In [70]:
import torch
import cv2
from models.experimental import attempt_load
from utils.augmentations import letterbox
from utils.general import check_img_size, non_max_suppression
from utils.torch_utils import select_device, time_sync
from utils.augmentations import letterbox

# Загрузить модель
device = select_device('')
model = attempt_load('yolov5s.pt', device=device)

# Загрузить изображение
img = cv2.imread('image.jpg')

# Копировать изображение для отображения
img_display = img.copy()

# Преобразовать изображение в формат, подходящий для модели
img = letterbox(img0, new_shape=640, auto=True, color=(0, 0, 0))[0]
img = img[:, :, ::-1].transpose(2, 0, 1) # BGR to RGB, to 3x416x416
img = np.ascontiguousarray(img)
img = torch.from_numpy(img).to(device)
img = img.float()  # uint8 to fp16/32
img /= 255.0  # 0 - 255 to 0.0 - 1.0
img = img[None]  

# Выполнить обнаружение объектов
pred = model(img)[0]
pred = non_max_suppression(pred, conf_thres=0.4, iou_thres=0.5)  # Изменить порог обнаружения на 0.2

# Отобразить bounding box на изображении
min_size = 32  # Минимальный размер bounding box
for i, det in enumerate(pred):
    if det is not None and len(det):
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], img_display.shape).round()
        for *xyxy, conf, cls in reversed(det):
            x1, y1, x2, y2 = map(int, xyxy)
            width = x2 - x1
            height = y2 - y1
            size = width * height
            min_size = min_size * min_size
            if size >= min_size:  # Проверить размер bounding box
                color = (0, 255, 0)  
                cv2.rectangle(img_display, (x1, y1), (x2, y2), color, 2)
            else:
                color = (0, 0, 255)  
                cv2.rectangle(img_display, (x1, y1), (x2, y2), color, 2)


# Отобразить изображение
cv2.imshow('Image', img_display)
cv2.waitKey(0)
cv2.destroyAllWindows()


YOLOv5  2024-4-11 Python-3.11.8 torch-2.2.2 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
